In [ ]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "./model/"
adapters_name = './output/checkpoint-10000/'

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /data/software/anaconda3/envs/huqiang/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /data/software/anaconda3/envs/huqiang/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/data/software/anaconda3/envs/huqiang/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /data/software/anaconda3/envs/huqiang did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


Starting to load the model ./qlora/model/ into memory


Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ./qlora/model/ and are newly initialized: ['model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq'

Successfully loaded the model ./qlora/model/ into memory


In [ ]:
tok.encode("A B C D")

[1, 319, 350, 315, 360]

In [ ]:
import random
from collections import Counter

def find_most_common(lst):
    
    counts = Counter(lst)

    max_count = max(counts.values())

    most_common_numbers = [num for num, count in counts.items() if count == max_count]

    result = random.choice(most_common_numbers)

    return result

In [ ]:
# !CUDA_LAUNCH_BLOCKING=1
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import torch
import json
from transformers import AutoTokenizer, AutoModelForMultipleChoice
from transformers.models.bert.configuration_bert import BertConfig
from sklearn.metrics import accuracy_score

# 检查GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

file_path = "./data/Q2_(2)_dev_data.json"
dev_dataset = []

with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        json_object = json.loads(line)
        dev_dataset.append(json_object)

# # Load your LLAMA model and tokenizer and move to GPU
# model_name = "./qlora/output/checkpoint-10000/"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# config = BertConfig.from_pretrained(model_name)
# model = AutoModelForMultipleChoice.from_pretrained(model_name, config=config).to(device)
model=m
tokenizer=tok

# Evaluation loop
true_labels = []
predicted_labels = []

ending_names = ["opa", "opb", "opc", "opd"]
for example in dev_dataset:
    # # Tokenize and prepare inputs
    # inputs = tokenizer([[example["question"], example[op]] for op in ending_names], return_tensors="pt", padding=True)
    # labels = torch.tensor(example["cop"]-1).unsqueeze(0).to(device)

    que=example["question"]
    opa=example["opa"]
    opb=example["opb"]
    opc=example["opc"]
    opd=example["opd"]
    input=f"Choose an answer from A, B, C, D for the question: {que}\nA: {opa}\nB: {opb}\nC: {opc}\nD: {opd}\nthe answer is"
    # text_segments = [f"{que} {example[op]}" for op in ending_names]
    # 使用标记器对文本进行编码
    inputs = tokenizer(input, return_tensors="pt", padding=True, truncation=True).to(device)

    # 使用模型进行评估
    with torch.no_grad():
        preds=[]
        for i in range(10):
            logits = model(**inputs).logits[:,-1,[319, 350, 315, 360]]
            predicted_class = logits.argmax().item()
            preds.append(predicted_class)
        predicted_class=find_most_common(preds)
        # print(logits.shape)

    # # Model inference
    # outputs = model(**{k: v.unsqueeze(0).to(device) for k, v in inputs.items()}, labels=labels)
    # logits = outputs.logits

    # Get predicted class
    # print(logits)
    

    # Record true and predicted labels
    true_labels.append(example["cop"]-1)
    predicted_labels.append(predicted_class)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Accuracy on dev set: {accuracy}")


Using device: cuda:0
Accuracy on dev set: 0.21324408319388


In [12]:
print(true_labels)
print("-------------------------")
print(predicted_labels)

[0, 0, 2, 2, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 3, 0, 0, 1, 2, 0, 0, 3, 1, 0, 3, 0, 2, 0, 1, 0, 0, 3, 2, 1, 3, 2, 2, 1, 0, 3, 3, 0, 1, 1, 0, 2, 2, 1, 2, 1, 0, 1, 2, 2, 0, 0, 3, 3, 2, 0, 3, 0, 3, 2, 3, 2, 1, 1, 2, 0, 1, 3, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 2, 0, 3, 0, 3, 1, 1, 1, 0, 2, 0, 1, 3, 1, 3, 0, 1, 0, 3, 1, 0, 3, 1, 2, 0, 0, 3, 0, 2, 0, 1, 2, 1, 0, 0, 2, 0, 0, 3, 1, 0, 0, 1, 3, 1, 2, 3, 1, 1, 1, 0, 1, 0, 2, 0, 2, 0, 0, 2, 1, 0, 2, 0, 3, 1, 0, 1, 2, 0, 1, 1, 0, 3, 2, 3, 1, 1, 1, 3, 0, 2, 3, 1, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 3, 3, 1, 0, 0, 0, 2, 2, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 3, 0, 3, 2, 1, 2, 2, 3, 2, 2, 3, 1, 1, 0, 3, 2, 2, 2, 0, 0, 3, 3, 0, 2, 1, 2, 3, 0, 0, 0, 1, 1, 0, 0, 3, 3, 3, 2, 3, 2, 2, 2, 3, 0, 3, 2, 3, 0, 0, 2, 0, 3, 3, 0, 3, 2, 0, 2, 3, 0, 0, 2, 2, 2, 3, 3, 1, 3, 0, 1, 1, 2, 2, 3, 0, 2, 2, 3, 3, 1, 1, 3, 0, 2, 2, 3, 2, 2, 2, 3, 1, 2, 3, 2, 0, 3, 3, 1, 0, 1, 1, 0, 0, 2, 1, 3, 0, 1, 1, 0, 0, 1, 1, 0, 3, 0, 0, 1, 0, 0, 0, 1, 0, 3, 3, 1, 3, 1, 1, 1, 1, 3, 2, 3, 3, 